In [1]:
import xmlrpc.client as xrpc

server = xrpc.ServerProxy("http://api.opensubtitles.org/xml-rpc")

In [2]:
credentials = server.LogIn("randomwalker", "sub1machine", "en", "MachineTitle")
token = credentials.get("token")
info = server.ServerInfo()
info['download_limits']['client_download_quota']

info


{'application': 'OpenSuber v0.2',
 'contact': 'admin@opensubtitles.org',
 'download_limits': {'client_24h_download_count': 6,
  'client_24h_download_limit': 200,
  'client_download_quota': 194,
  'client_ip': '139.179.227.115',
  'global_24h_download_limit': 200,
  'limit_check_by': 'user_ip'},
 'last_update_strings': {'ar': '2007-02-03 21:36:14',
  'at': '2016-02-28 01:41:54',
  'bg': '2007-02-03 21:36:14',
  'br': '2016-02-25 05:52:22',
  'ca': '2014-11-13 11:32:44',
  'cs': '2014-12-25 01:00:17',
  'da': '2015-06-16 14:25:51',
  'de': '2014-11-13 05:01:32',
  'el': '2015-09-07 11:58:19',
  'en': '2014-11-12 12:34:41',
  'eo': '2014-11-13 18:27:49',
  'es': '2014-11-20 13:11:05',
  'et': '2014-11-12 16:31:04',
  'eu': '2014-11-13 17:08:41',
  'fa': '2007-02-03 21:36:16',
  'fi': '2014-11-12 13:16:05',
  'fr': '2014-11-13 08:02:00',
  'gl': '2007-10-11 16:23:40',
  'he': '2007-02-03 21:36:17',
  'hi': '2016-09-08 15:51:39',
  'hr': '2014-11-13 01:08:44',
  'hu': '2014-11-13 12:00:33',

In [3]:
# server.GetSubLanguages()
# server.SearchMoviesOnIMDB(token, "troy")
subtitles = server.SearchSubtitles(token, [{'imdbid': '1515091', 'sublanguageid': 'eng'}])

# impaired_support = True

# subtitles = list(filter(lambda sub: sub['SubHearingImpaired'] == '1', subtitles))
subtitles

{'data': [{'IDMovie': '54856',
   'IDMovieImdb': '1515091',
   'IDSubMovieFile': '0',
   'IDSubtitle': '4474095',
   'IDSubtitleFile': '1953036639',
   'ISO639': 'en',
   'LanguageName': 'English',
   'MatchedBy': 'imdbid',
   'MovieByteSize': '0',
   'MovieFPS': '25.000',
   'MovieHash': '0',
   'MovieImdbRating': '7.5',
   'MovieKind': 'movie',
   'MovieName': 'Sherlock Holmes: A Game of Shadows',
   'MovieNameEng': '',
   'MovieReleaseName': 'Harks Re-Encode x264 Updated Complete',
   'MovieTimeMS': '0',
   'MovieYear': '2011',
   'QueryNumber': '0',
   'QueryParameters': {'imdbid': '1515091', 'sublanguageid': 'eng'},
   'SeriesEpisode': '0',
   'SeriesIMDBParent': '0',
   'SeriesSeason': '0',
   'SubActualCD': '1',
   'SubAddDate': '2012-02-20 11:21:34',
   'SubAuthorComment': '',
   'SubBad': '0',
   'SubComments': '0',
   'SubDownloadLink': 'http://dl.opensubtitles.org/en/download/src-api/vrf-19a30c56/sid-K5SIpeJ8Fn1Oax3-OrZUx7vidC1/file/1953036639.gz',
   'SubDownloadsCnt': '212

In [4]:
# subtitle_ids = [ sub['IDSubtitleFile'] for sub in subtitles]
# result = server.DownloadSubtitles(token, subtitle_ids[:10])

# result

In [5]:
# print([res['idsubtitlefile'] for res in result['data']])
# # print(subtitle_ids[:10])


# import base64
# import gzip

# with open("/home/burak/denem.txt", 'w') as file:
#     file.write(gzip.decompress(base64.b64decode(result['data'][0]['data'])).decode())


In [14]:
subtitles_path = "/Users/mesutgurlek/Documents/Machine Learning/project/Movie-Category-Classification-from-Subtitles/Subtitles"
url_template = "http://www.imdb.com/search/title?genres=%s&explore=genres&sort=num_votes,desc&view=simple"
imdb_page_limit = 1

server = xrpc.ServerProxy("http://api.opensubtitles.org/xml-rpc")
token = server.LogIn("randomwalker", "sub1machine", "en", "MachineTitle").get("token")
remaining_quota = server.ServerInfo()['download_limits']['client_download_quota']

print(server.ServerInfo())

categories = ['action', 'comedy', 'horror', 'war', 'romance', 'adventure']
subtitle_per_category = int(remaining_quota / len(categories))



def parse_movies(imdb_ids=['0172495'], category_name='War'):
        # movie_links = response.css(".bnone").xpath('@href').extract()
        # for link in movie_links:
        #     yield scrapy.Request(url=response.urljoin(link), callback=self.parse_movie)
        print("Listing scraped IMDB ids for%s: ", category_name)
        print(imdb_ids)
        impaired_support = True

        subtitles = {} # key => IDSubtitleFile, value => Metadata of subtitle

        # USING IMDB ID'S, DOWNLOAD THEIR METADATA AND SELECT ID OF A SUITABLE SUBTITLE FOR EACH MOVIE
        for imdb_id in imdb_ids[:subtitle_per_category]:
            print("Searching subtitle for movie with ID: %s" % imdb_id)
            found_subtitles = server.SearchSubtitles(token, [{'imdbid': imdb_id, 'sublanguageid': 'eng'}])['data']

            print("!!!!! Found subtitles: %s" % found_subtitles)

            if impaired_support:
                impaired_subtitles = list(filter(lambda sub: sub['SubHearingImpaired'] == '1', found_subtitles))

            impaired_label = ""
            if len(impaired_subtitles) > 0:
                subtitle = impaired_subtitles[0]
                impaired_label = "(IMPAIRED)"
            else:
                subtitle = found_subtitles[0] # for now get the first subtitle

            filename = "%s/%s/%s %s" % (subtitles_path, category_name, subtitle['MovieName'], impaired_label)

            subtitles[subtitle['IDSubtitleFile']] = {'imdb_id': imdb_id, 'filename': filename,
                                                     'movie_name': subtitle['MovieName'],
                                                    'SubDownloadLink': subtitle['SubDownloadLink']}

            # with open(filename, 'w') as f:
            #     f.write(json.dumps(subtitle['SubDownloadLink']))
            #     # yield parse_movie(imdb_id)


        #DOWNLOAD SUBTITLES AND WRITE THEM INTO FILES
        print("Downloading subtitles of %s" % category_name)
        subtitle_ids = [ idsubtitlefile for idsubtitlefile, val in subtitles.items()]
        subtitle_files_response = server.DownloadSubtitles(token, subtitle_ids)

        import base64
        import gzip

        if subtitle_files_response['status'] == '200 OK':
            print("Subtitles downloaded, writing to files...")
            for subtitle_object in subtitle_files_response['data']: #each subtitle_object has base64 data and idsubtitlefile key
                sub = subtitles[subtitle_object['idsubtitlefile']]

                with open(sub['filename'], 'w', encoding='utf-8', errors='ignore') as file:
                    
                    print(type(gzip.decompress(base64.b64decode(subtitle_object['data'])).decode(encoding='UTF-16BE')))
                    
                    # file.write(gzip.decompress(base64.b64decode(subtitle_object['data'])).decode()
                    print("Subtitle file saved into: %s" % sub['filename'])


parse_movies(imdb_ids=['0172495'])

                    

{'seconds': 0.003, 'xmlrpc_version': '0.1', 'users_online_total': 13396, 'users_max_alltime': '27449', 'users_registered': '3399216', 'subs_downloads': '4294967295', 'contact': 'admin@opensubtitles.org', 'users_loggedin': 679, 'website_url': 'http://www.opensubtitles.org', 'subs_subtitle_files': '3923723', 'download_limits': {'client_24h_download_count': 6, 'client_24h_download_limit': 200, 'limit_check_by': 'user_ip', 'client_ip': '139.179.227.115', 'client_download_quota': 194, 'global_24h_download_limit': 200}, 'last_update_strings': {'bg': '2007-02-03 21:36:14', 'de': '2014-11-13 05:01:32', 'sq': '2014-11-13 09:13:12', 'hr': '2014-11-13 01:08:44', 'hu': '2014-11-13 12:00:33', 'ja': '2015-01-20 19:14:48', 'cs': '2014-12-25 01:00:17', 'el': '2015-09-07 11:58:19', 'zh': '2014-11-25 02:32:33', 'da': '2015-06-16 14:25:51', 'km': '2014-11-27 04:29:35', 'en': '2014-11-12 12:34:41', 'is': '2014-11-19 00:24:35', 'sk': '2014-11-25 04:07:58', 'et': '2014-11-12 16:31:04', 'hy': '2016-09-15 19:

UnicodeDecodeError: 'utf-16-be' codec can't decode byte 0x0a in position 114290: truncated data